In [2]:
import os
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import MinMaxScaler

In [7]:
import os
import pandas as pd
import numpy as np

# Path to frame directory (update if needed)
frames_dir = "data/images"
frame_files = sorted([f for f in os.listdir(frames_dir) if f.startswith("Frame_") and f.endswith(".csv")])

# Initialize list to store features
feature_list = []

# Debug: Check how many frames are found
print(f"Total Frames Found: {len(frame_files)}")

for file_name in frame_files:
    file_path = os.path.join(frames_dir, file_name)
    frame_data = pd.read_csv(file_path, header=None)

    # Debug: Print the first few frame values
    print(f"🔍 Processing {file_name} - Shape: {frame_data.shape}")

    if frame_data.isna().sum().sum() > 0:
        print(f"⚠️ Warning: {file_name} contains NaN values!")

    if frame_data.shape[0] == 0 or frame_data.shape[1] == 0:
        print(f"🚨 Skipping {file_name} - Empty frame!")
        continue  # Skip empty frames

    # Convert frame to NumPy array
    frame_np = frame_data.to_numpy()

    # Compute statistical features (ignoring NaNs)
    min_temp = np.nanmin(frame_np) if np.isnan(frame_np).sum() < frame_np.size else np.nan
    max_temp = np.nanmax(frame_np) if np.isnan(frame_np).sum() < frame_np.size else np.nan
    mean_temp = np.nanmean(frame_np) if np.isnan(frame_np).sum() < frame_np.size else np.nan
    std_temp = np.nanstd(frame_np) if np.isnan(frame_np).sum() < frame_np.size else np.nan
    median_temp = np.nanmedian(frame_np) if np.isnan(frame_np).sum() < frame_np.size else np.nan
    q1 = np.nanpercentile(frame_np, 25) if np.isnan(frame_np).sum() < frame_np.size else np.nan
    q3 = np.nanpercentile(frame_np, 75) if np.isnan(frame_np).sum() < frame_np.size else np.nan
    iqr = q3 - q1 if np.isnan(frame_np).sum() < frame_np.size else np.nan
    skewness = pd.DataFrame(frame_np.flatten()).skew()[0] if np.isnan(frame_np).sum() < frame_np.size else np.nan
    kurtosis = pd.DataFrame(frame_np.flatten()).kurt()[0] if np.isnan(frame_np).sum() < frame_np.size else np.nan
    high_temp_pixels = np.sum(frame_np > np.nanmean(frame_np) + 1.5 * np.nanstd(frame_np)) if np.isnan(frame_np).sum() < frame_np.size else 0

    # Append extracted features
    feature_list.append([file_name, min_temp, max_temp, mean_temp, std_temp, median_temp, q1, q3, iqr, skewness, kurtosis, high_temp_pixels])

# Create DataFrame
feature_columns = ["Frame", "Min_Temp", "Max_Temp", "Mean_Temp", "Std_Temp", "Median_Temp", "Q1", "Q3", "IQR", "Skewness", "Kurtosis", "High_Temp_Pixels"]
df_features = pd.DataFrame(feature_list, columns=feature_columns)

# Debug: Show the first few extracted features
print(df_features.head())

# Save as CSV
df_features.to_csv("thermal_frame_features.csv", index=False)
print("✅ Debug Feature extraction completed. Check 'thermal_frame_features.csv'.")


Total Frames Found: 1564
🔍 Processing Frame_1.csv - Shape: (201, 202)
⚠️ Warning: Frame_1.csv contains NaN values!
🔍 Processing Frame_10.csv - Shape: (201, 202)
⚠️ Warning: Frame_10.csv contains NaN values!
🔍 Processing Frame_100.csv - Shape: (201, 202)
⚠️ Warning: Frame_100.csv contains NaN values!
🔍 Processing Frame_1000.csv - Shape: (201, 202)
⚠️ Warning: Frame_1000.csv contains NaN values!
🔍 Processing Frame_1001.csv - Shape: (201, 202)
⚠️ Warning: Frame_1001.csv contains NaN values!
🔍 Processing Frame_1002.csv - Shape: (201, 202)
⚠️ Warning: Frame_1002.csv contains NaN values!
🔍 Processing Frame_1003.csv - Shape: (201, 202)
⚠️ Warning: Frame_1003.csv contains NaN values!
🔍 Processing Frame_1004.csv - Shape: (201, 202)
⚠️ Warning: Frame_1004.csv contains NaN values!
🔍 Processing Frame_1005.csv - Shape: (201, 202)
⚠️ Warning: Frame_1005.csv contains NaN values!
🔍 Processing Frame_1006.csv - Shape: (201, 202)
⚠️ Warning: Frame_1006.csv contains NaN values!
🔍 Processing Frame_1007.csv